In [7]:
import pandas as pd
import requests
import base64
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [14]:
# Load homology data from MMseqs2 results
yih_homologs = pd.read_csv('yih_genes_mmseq.tsv', sep='\t')

In [15]:
yih_homologs.head()

,query,target,identity,alignment_length,mismatches,gap_openings,query_start,query_end,target_start,target_end,evalue,bit_score,genome_id
0,AYG21329.1|CP032667.1,CP018976.1_cds_AQV54885.1_3704,1.000,690,0,0,1,690,1,690,1.900000e-158,489,GCA_002012045.1
1,AYG21329.1|CP032667.1,CP018976.1_cds_AQV54885.1_3704,0.866,213,28,0,612,400,612,400,6.455000e-37,130,GCA_002012045.1
2,AYG21329.1|CP032667.1,CP018976.1_cds_AQV54885.1_3704,0.975,123,3,0,41,163,41,163,1.682000e-22,87,GCA_002012045.1
3,AYG21329.1|CP032667.1,CP018976.1_cds_AQV54885.1_3704,0.971,105,3,0,317,213,317,213,6.730000e-18,74,GCA_002012045.1
4,AYG21329.1|CP032667.1,CP018976.1_cds_AQV54885.1_3704,0.917,102,8,0,104,3,104,3,4.892000e-16,68,GCA_002012045.1


In [8]:
def download_github_folder(repo, folder_path, local_dir):
    """
    Download a folder from a GitHub repo
    """
    api_url = f"https://api.github.com/repos/{repo}/contents/{folder_path}"
    response = requests.get(api_url)
    response.raise_for_status()
    
    os.makedirs(local_dir, exist_ok=True)
    
    for item in response.json():
        if item['type'] == 'file':
            file_response = requests.get(item['download_url'])
            file_path = os.path.join(local_dir, item['name'])
            
            with open(file_path, 'wb') as f:
                f.write(file_response.content)
            print(f"Downloaded: {item['name']}")
        elif item['type'] == 'dir':
            subfolder_path = os.path.join(local_dir, item['name'])
            download_github_folder(repo, f"{folder_path}/{item['name']}", subfolder_path)


In [ ]:
# Downloads genomic data from GitHub repository (ShigellaProject)
# If not downloaded, get genomic fna (will take some time to download) 
repo = "zsfrbkv/ShigellaProject"
folder_path = "1Tree/Data/Genomes_fna"
local_dir = "Genomes_fna"
download_github_folder(repo, folder_path, local_dir)

In [ ]:
# Extracts genomic coordinates and strand information from FASTA descriptions
# Create mapping target2info (will take some time to pase fasta files)
fna_dir = local_dir
genome_id_lst = yih_homologs.genome_id.unique()
import glob
target2info = {}
for genome_id in genome_id_lst:
    fna_pattern = os.path.join(fna_dir, f'{genome_id}*.fna')
    fna_files = glob.glob(fna_pattern)
    
    if fna_files:
        fna_f = fna_files[0]
        target2info.update({r.id.replace('lcl|', ''): r.description
              for r in SeqIO.parse(fna_f, 'fasta')})
    else:
        print(f"Warning: No .fna file found for genome_id: {genome_id}")

In [ ]:
# Maps protein IDs to gene names 
protein_queryGene = dict(zip(['AYG21329.1|CP032667.1', 'AYG21328.1|CP032667.1',
       'AYG21327.1|CP032667.1', 'AYG21326.1|CP032667.1',
       'AYG21325.1|CP032667.1', 'AYG21324.1|CP032667.1',
       'AYG21323.1|CP032667.1', 'AYG21322.1|CP032667.1',
       'AYG21321.1|CP032667.1', 'AYG21320.1|CP032667.1'],
        ['ompL', 'yihO', 'yihP', 'yihQ', 
        'yihR', 'yihS', 'yihT', 'yihU', 'yihV', 'yihW'])) 

yih_homologs['query_gene'] = yih_homologs['query'].map(protein_queryGene)
yih_homologs['target_info'] = yih_homologs.target.map(target2info)
yih_homologs['target_location'] = yih_homologs.target_info.str.extract(r'\[location=([^\]]+)\]')
yih_homologs['strand'] = yih_homologs['target_location'].apply(lambda x: '-' if 'complement' in x else '+')
yih_homologs[['start', 'end']] = yih_homologs['target_location'].\
apply(lambda x: x.replace('complement(', '').replace(')', '')).\
astype(str).str.split('\.\.', expand=True).astype(int)

yih_homologs = yih_homologs[yih_homologs.groupby(['target', 'query'])['bit_score'].\
transform(max) == yih_homologs['bit_score']]
yih_homologs['locus_tag'] = yih_homologs.target_info.str.extract(r'\[locus_tag=([^\]]+)\]')
yih_homologs['chr_id'] = yih_homologs.target.str.split('_', expand=True)[0]

In [11]:
protein_queryGene = dict(zip(['AYG21329.1|CP032667.1', 'AYG21328.1|CP032667.1',
       'AYG21327.1|CP032667.1', 'AYG21326.1|CP032667.1',
       'AYG21325.1|CP032667.1', 'AYG21324.1|CP032667.1',
       'AYG21323.1|CP032667.1', 'AYG21322.1|CP032667.1',
       'AYG21321.1|CP032667.1', 'AYG21320.1|CP032667.1'],
        ['ompL', 'yihO', 'yihP', 'yihQ', 
        'yihR', 'yihS', 'yihT', 'yihU', 'yihV', 'yihW'])) 

def get_colocalization(blastp_drop,
                       map_query_gene=protein_queryGene, 
                       threshold=5000):
    # blastp_drop = pd.read_csv(args.blastpOutputExt, sep='\t')
    # blastp_drop['query_gene'] = blastp_drop.query_protein.map(map_query_gene)

    # Compute midpoint coordinates
    blastp_drop['midpoint'] = (blastp_drop.start + blastp_drop.end) / 2
    
    # Sort by chromosome and midpoint
    blastp_drop_sorted = blastp_drop.groupby("chr_id").apply(
        lambda x: x.sort_values("midpoint")).reset_index(drop=True)
    
    # Find locus composition
    dict_assembly_queryGenes = {}
    dict_assembly_locusTags = {}
    dict_assembly_coords = {}
    
    for chr_id, group in blastp_drop_sorted.groupby("chr_id"):
        midpoints = group.midpoint.values
        locusTags = group.locus_tag.values
        queryGenes = group.query_gene.values
        
        # Initialize first locus
        current_loci = {
            'queryGenes': [queryGenes[0]],
            'locusTags': [str(locusTags[0])],
            'coords': [str(midpoints[0])]
        }
        
        queryGenes_loci = []
        locusTags_loci = []
        coords_loci = []
        
        for k in range(len(midpoints) - 1):
            if (midpoints[k + 1] - midpoints[k]) <= threshold:
                # Add to current locus
                current_loci['queryGenes'].append(queryGenes[k + 1])
                current_loci['locusTags'].append(str(locusTags[k + 1]))
                current_loci['coords'].append(str(midpoints[k + 1]))
            else:
                # Save current locus and start new one
                queryGenes_loci.append('/'.join(current_loci['queryGenes']))
                locusTags_loci.append('/'.join(current_loci['locusTags']))
                coords_loci.append('/'.join(current_loci['coords']))
                
                # Start new locus
                current_loci = {
                    'queryGenes': [queryGenes[k + 1]],
                    'locusTags': [str(locusTags[k + 1])],
                    'coords': [str(midpoints[k + 1])]
                }
        
        # Add the last locus
        queryGenes_loci.append('/'.join(current_loci['queryGenes']))
        locusTags_loci.append('/'.join(current_loci['locusTags']))
        coords_loci.append('/'.join(current_loci['coords']))
        
        # Join all loci for this chromosome
        dict_assembly_queryGenes[chr_id] = ' '.join(queryGenes_loci)
        dict_assembly_locusTags[chr_id] = ' '.join(locusTags_loci)
        dict_assembly_coords[chr_id] = ' '.join(coords_loci)
    
    # Create mapping dictionaries
    queryGenes_list = list(map_query_gene.values())
    dict_locusTag_queryGeneBlock = {}
    dict_locusTag_locusTagBlock = {}
    
    for chr_id in blastp_drop_sorted.chr_id.unique():
        locusTag_blocks = dict_assembly_locusTags[chr_id].split()
        gene_blocks = dict_assembly_queryGenes[chr_id].split()
        
        for locusTag_block, gene_block in zip(locusTag_blocks, gene_blocks):
            locus_tags = locusTag_block.split('/')
            genes = gene_block.split('/')
            
            # Determine order based on queryGenes_list
            if (queryGenes_list.index(genes[0]) > 
                queryGenes_list.index(genes[-1])):
                gene_block = '/'.join(reversed(genes))
                locusTag_block = '/'.join(reversed(locus_tags))
            
            for locus_tag in locus_tags:
                dict_locusTag_queryGeneBlock[locus_tag] = gene_block
                dict_locusTag_locusTagBlock[locus_tag] = locusTag_block
    
    # Add locus information columns
    blastp_drop_sorted['queryGenes_locus'] = blastp_drop_sorted.locus_tag.map(
        dict_locusTag_queryGeneBlock)
    blastp_drop_sorted['locusTag_locus'] = blastp_drop_sorted.locus_tag.map(
        dict_locusTag_locusTagBlock)
    return blastp_drop_sorted

In [ ]:
# Will take some time
# Identifies colocalized genes using a proximity threshold (5kb default)
yih_homologs_loci = get_colocalization(yih_homologs)

In [ ]:
yih_homologs_loci['set_queryGenes_locus'] = yih_homologs_loci.\
queryGenes_locus.apply(lambda x: '~'.join(sorted(set(x.split('/')))))
yih_homologs_loci = yih_homologs_loci[yih_homologs_loci.set_queryGenes_locus.apply(lambda x: 'yihV' in x and 'yihT' in x)]
yih_homologs_loci = yih_homologs_loci.drop_duplicates('target')

yih_homologs_loci['start_min'] = yih_homologs_loci.groupby('chr_id')['start'].transform(min)
yih_homologs_loci['end_max'] = yih_homologs_loci.groupby('chr_id')['end'].transform(max)
yih_homologs_loci['set_locusTag_locus'] = yih_homologs_loci['locusTag_locus'].\
apply(lambda x: '~'.join(sorted(set(x.split('/')))))

# Classifies operon variants: "long" variant: contains yihQ or yihS genes
# "short" variant: missing these genes
yih_homologs_loci['variant'] = yih_homologs_loci.set_queryGenes_locus.apply(lambda x: 
                                             'long' if 'yihQ' in x or 'yihS' in x else 'short')

In [ ]:
## Save to dataframe
## yih_homologs_loci.to_csv('yih_homologs_loci.csv', sep='\t', index=False)

In [ ]:
## ======= Create target2info data ========#
target2info_data = pd.DataFrame({'target': [k for k in target2info],
             'target_info': [target2info[k] for k in target2info]})

target2info_data['target_location'] = target2info_data.target_info.str.extract(r'\[location=([^\]]+)\]')
target2info_data['strand'] = target2info_data['target_location'].apply(lambda x: '-' if 'complement' in x else '+')

target2info_data = target2info_data[~target2info_data.target_location.str.contains('join')]
target2info_data = target2info_data[~target2info_data.target_location.str.contains('<')]
target2info_data = target2info_data[~target2info_data.target_location.str.contains('>')]


target2info_data[['start', 'end']] = target2info_data['target_location'].\
apply(lambda x: x.replace('complement(', '').replace(')', '')).\
astype(str).str.split('\.\.', expand=True).astype(int)


target2info_data['locus_tag'] = target2info_data.target_info.str.extract(r'\[locus_tag=([^\]]+)\]')

target2info_data['chr_id'] = target2info_data.target.str.split('_', expand=True)[0]

In [ ]:
## Save the dataframe
#target2info_data.to_csv('target2info_data_altToGFFs.csv', index=False, sep='\t')